# Sense from Numbers

## Configuring CITE libraries for almond kernel

First, we'll make a bintray repository with CITE libraries available to your almond kernel.

In [1]:
val myBT = coursierapi.MavenRepository.of("https://dl.bintray.com/neelsmith/maven")

interp.repositories() ++= Seq(myBT)

myBT: coursierapi.MavenRepository = MavenRepository(https://dl.bintray.com/neelsmith/maven)

Next, we bring in specific libraries from the new repository using almond's `$ivy` magic:

In [2]:
import $ivy.`edu.holycross.shot::ohco2:10.16.0`
import $ivy.`edu.holycross.shot.cite::xcite:4.1.1`
import $ivy.`edu.holycross.shot::scm:7.2.0`
import $ivy.`edu.holycross.shot::dse:5.2.2`
import $ivy.`edu.holycross.shot::citebinaryimage:3.1.1`
import $ivy.`edu.holycross.shot::citeobj:7.3.4`
import $ivy.`edu.holycross.shot::citerelations:2.5.2`
import $ivy.`edu.holycross.shot::cex:6.3.3`

import $ivy.`org.plotly-scala::plotly-almond:0.7.1`
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

// if you want to have the plots available without an internet connection:
// init(offline=true)

// restrict the output height to avoid scrolling in output cells
repl.pprinter() = repl.pprinter().copy(defaultHeight = 3)

import $ivy.$                                  

import $ivy.$                                     

import $ivy.$                              

import $ivy.$                              

import $ivy.$                                          

import $ivy.$                                  

import $ivy.$                                        

import $ivy.$                              


import $ivy.$                                      

import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

// if you want to have the plots available without an internet connection:
// init(offline=true)

// restrict the output height to avoid scrolling in output cells


## Imports

From this point on, your notebook consists of completely generic Scala, with the CITE Libraries available to use.

In [3]:
// Import some CITE libraries
import edu.holycross.shot.cite._
import edu.holycross.shot.ohco2._
import edu.holycross.shot.scm._
import edu.holycross.shot.citeobj._
import edu.holycross.shot.citerelation._
import edu.holycross.shot.dse._
import edu.holycross.shot.citebinaryimage._
import edu.holycross.shot.ohco2._

import almond.display.UpdatableDisplay
import almond.interpreter.api.DisplayData.ContentType
import almond.interpreter.api.{DisplayData, OutputHandler}

import java.io.File
import java.io.PrintWriter

import scala.io.Source

import java.text.SimpleDateFormat
import java.util.Date


import edu.holycross.shot.cite._

import edu.holycross.shot.ohco2._

import edu.holycross.shot.scm._

import edu.holycross.shot.citeobj._

import edu.holycross.shot.citerelation._

import edu.holycross.shot.dse._

import edu.holycross.shot.citebinaryimage._

import edu.holycross.shot.ohco2._


import almond.display.UpdatableDisplay

import almond.interpreter.api.DisplayData.ContentType

import almond.interpreter.api.{DisplayData, OutputHandler}


import java.io.File

import java.io.PrintWriter


import scala.io.Source


import java.text.SimpleDateFormat

import java.util.Date


## Useful Functions

Save a string to a names file:

In [4]:
def saveString(s:String, filePath:String = "", fileName:String = "temp.txt"):Unit = {
		 val writer = new PrintWriter(new File(s"${filePath}${fileName}"))
         writer.write(s)
         writer.close()
	}

defined function saveString

Convert a Roman Numeral to an Integer:

In [5]:
def fromRoman(s: String) : Int = {
	try {
		val numerals = Map('I' -> 1, 'V' -> 5, 'X' -> 10, 'L' -> 50, 'C' -> 100, 'D' -> 500, 'M' -> 1000)

		s.toUpperCase.map(numerals).foldLeft((0,0)) {
		  case ((sum, last), curr) =>  (sum + curr + (if (last < curr) -2*last else 0), curr) }._1
	} catch {
		case e:Exception => throw new Exception(s""" "${s}" is not a valid Roman Numeral.""")
	}
}

defined function fromRoman

Like `.split`, but preserving the character we split on:

In [6]:
def splitWithSplitter(text: String, puncs: String): Vector[String] = {
	//val regexWithSplitter = s"((?<=${puncs})|(?=${puncs}))"
    val regexWithSplitter = s"((?<=${puncs}))"
	text.split(regexWithSplitter).toVector.filter(_.size > 0)
}

defined function splitWithSplitter

Pretty Print Things:

In [7]:
def showMe(v:Any):Unit = {
  v match {
    case _:StringHistogram => {
        for ( h <- v.asInstanceOf[StringHistogram].histogram ) {
            println(s"${h.count}\t${h.s}")
        }
    }
  	case _:Corpus => {
  		for ( n <- v.asInstanceOf[Corpus].nodes) {
  			println(s"${n.urn.passageComponent}\t\t${n.text}")
  		}	
  	}
    case _:Vector[Any] => println(s"""\n----\n${v.asInstanceOf[Vector[Any]].mkString("\n")}\n----\n""")
    case _:Iterable[Any] => println(s"""\n----\n${v.asInstanceOf[Iterable[Any]].mkString("\n")}\n----\n""")
    case _ => println(s"\n-----\n${v}\n----\n")
  }
}

defined function showMe

Date Stuff

## Load Some Data

Load up-to-date data from the NY Times. Source: <https://github.com/nytimes/covid-19-data>.

In [8]:
val dataLines: Vector[String] = {
    scala.io.Source.fromURL("https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv").mkString.split("\n").toVector
}

// quick test
val badLines = dataLines.filter( l => {
    l.split(",").size != 5
})

assert ( badLines.size == 0 )

dataLines: Vector[String] = Vector(
  "date,state,fips,cases,deaths",
...
badLines: Vector[String] = Vector()

## Make Data Structures

For State data:

In [9]:
case class StateDatum( date: String, state: String, cases: Int, deaths: Int)

defined class StateDatum

For Aggregate data:

In [10]:
case class RunningTally( days: Vector[StateDatum] )

defined class RunningTally

A daily snapshot (for the USA or a single state)

In [11]:
case class DailySnapshot( date: String, newCases: Int, newDeaths: Int)

defined class DailySnapshot

## Load Data

In [12]:

val vst: Vector[StateDatum] = dataLines.tail.map( dl => {
    val fields: Vector[String] = dl.split(",").toVector
    val date: String = fields(0)
   val state: String = fields(1)
    val cases: Int = fields(3).toInt
    val deaths: Int = fields(4).toInt
    StateDatum(date, state, cases, deaths)
})

val rt = RunningTally(vst)


vst: Vector[StateDatum] = Vector(
  StateDatum("2020-01-21", "Washington", 1, 0),
...
rt: RunningTally = RunningTally(
  Vector(
...

## Comparative Data

Source for USA Data: [“Deaths and Mortality”, CDC](https://www.cdc.gov/nchs/fastats/deaths.htm).

Source for South Carolina Data: [“Stats of the State of South Carolina”, CDC](https://www.cdc.gov/nchs/pressroom/states/southcarolina/southcarolina.htm)

In [13]:
val usa2017TotalDeaths: Int = 2813502
val usa2017DailyDeaths: Int = usa2017TotalDeaths / 365

// Chronic lower respiratory diseases
val usa2017TotalDeathsCLRD: Int = 160201
val usa2017DailyDeathsCLRD: Int = 160201 / 365

// Influenza and Pneumonia
val usa2017TotalDeathsIPn: Int = 55672
val usa2017DailyDeathsIPn: Int = 55672 / 365

// All respiratory-related deaths
val usa2017TotalDeathsResp: Int = usa2017TotalDeathsCLRD + usa2017TotalDeathsIPn
val usa2017DailyDeathsResp: Int = usa2017TotalDeathsResp / 365

// All respiratory-related deaths, 6-month Flu Season
val usa2017DailyDeathsResp6MoFlu: Int = {
    usa2017DailyDeathsCLRD + (usa2017TotalDeathsIPn / 182 )
}

// Suicide
val usa2017SuicideTotal: Int = 47173
val usa2017SuicideDaily: Int = usa2017SuicideTotal / 365

// Heart Disease
val usa2017HeartTotal: Int = 647457
val usa2017HeartDaily: Int = usa2017HeartTotal / 365

// Accidents
val usa2017AccidentTotal: Int = 169936
val usa2017AccidentDaily: Int = usa2017AccidentTotal / 365

// Cancer
val usa2017CancerTotal: Int = 599108
val usa2017CancerDaily: Int = usa2017CancerTotal / 365



usa2017TotalDeaths: Int = 2813502
usa2017DailyDeaths: Int = 7708
usa2017TotalDeathsCLRD: Int = 160201
usa2017DailyDeathsCLRD: Int = 438
usa2017TotalDeathsIPn: Int = 55672
usa2017DailyDeathsIPn: Int = 152
usa2017TotalDeathsResp: Int = 215873
usa2017DailyDeathsResp: Int = 591
usa2017DailyDeathsResp6MoFlu: Int = 743
usa2017SuicideTotal: Int = 47173
usa2017SuicideDaily: Int = 129
usa2017HeartTotal: Int = 647457
usa2017HeartDaily: Int = 1773
usa2017AccidentTotal: Int = 169936
usa2017AccidentDaily: Int = 465
usa2017CancerTotal: Int = 599108
usa2017CancerDaily: Int = 1641

## Data Functions

In [14]:
def totalNewDaily(data: Vector[StateDatum] = rt.days, state: Option[String] = None): Vector[DailySnapshot] = {
    val sortedData: Vector[StateDatum] = {
        val filtered: Vector[StateDatum] = {
            state match {
                case Some(s) => data.filter(_.state == s)
                case None => {
                    val groupedByDay: Vector[(String, Vector[StateDatum])] = {
                        data.groupBy(_.date).toVector
                    }
                    val merged: Vector[StateDatum] = groupedByDay.map( gbd => {
                        val vec = gbd._2
                        val vecDate: String = vec.head.date
                        val vecState: String = "USA"
                        val vecCases: Int = vec.map(_.cases).sum
                        val vecDeaths: Int = vec.map(_.deaths).sum
                        StateDatum(vecDate, vecState, vecCases, vecDeaths)
                    })
                    merged
                }
            }
        }
        val sorted: Vector[StateDatum] = filtered.sortBy(_.date)
        sorted
    }
    // We don't want a running, cumulative tally, but new cases/deaths each day
    sortedData.zipWithIndex.map( sd => {
        val d: StateDatum = sd._1
        val i: Int = sd._2
        val newCases: Int = {
            if (i == 0) d.cases
            else {
                val totalToday: Int = d.cases
                val totalPrev: Int = {
                    sortedData(i-1).cases
                }
                totalToday - totalPrev
            }
        }
        val newDeaths: Int = {
            if (i == 0) d.deaths
            else {
                val totalToday: Int = d.deaths
                val totalPrev: Int = {
                    sortedData(i-1).deaths
                }
                totalToday - totalPrev
            }
        }
        DailySnapshot(d.date, newCases, newDeaths)
    })
    
}

defined function totalNewDaily

## The Pure “Curve”

Just 2020 COVID-19 Deaths, as reported by the New York Times (<https://github.com/nytimes/covid-19-data>).

In [15]:
val deaths: Seq[Int] = totalNewDaily(state = None).map(_.newDeaths).toSeq

val howManyDays: Int = deaths.size
val ipnBase: Seq[Int] = (1 to howManyDays).map( d => usa2017DailyDeathsResp)


val trace3 = Scatter(
  (1 to howManyDays),
  deaths,
  name = "2020 Covid Daily Deaths",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(204, 204, 204, 0.95),
    line = Line(
      color = Color.RGBA(217, 217, 217, 1.0),
      width = 1.0
    ),
    symbol = Symbol.Circle(),
    size = 3
  )
)

val data = Seq(trace3)

val layout = Layout("Daily USA COVID Deaths")

plot(data, layout)

deaths: Seq[Int] = Vector(
  0,
...
howManyDays: Int = 77
ipnBase: Seq[Int] = Vector(
  591,
...
trace3: Scatter = Scatter(
  Some(
...
data: Seq[Scatter] = List(
  Scatter(
...
layout: Layout = Layout(
  Some("Daily USA COVID Deaths"),
...
res14_6: String = "plot-b2736625-ebed-4c65-9345-1b6c1a587b85"

## USA Plot

Plotting daily deaths from COVID-19, as reported by the New York Times (<https://github.com/nytimes/covid-19-data>). The comparison is an aggregate of two CDC categories for US deaths: "Chronic Respiratory Illness" and "Influenza and Pneumonia". Annual numbers in these two categories are added, and divided by 365 for a daily number (see code above). 

Also displayed is a second comparison line, in which "Influenza and Pneumonia" daily rates are calculated based on a 6-month Flu Season (October–March, as the CDC does it).

In [16]:
val deaths: Seq[Int] = totalNewDaily(state = None).map(_.newDeaths).toSeq

val howManyDays: Int = deaths.size
val ipnBase: Seq[Int] = (1 to howManyDays).map( d => usa2017DailyDeathsResp)
val ipnBase6Mo: Seq[Int] = (1 to howManyDays).map( d => usa2017DailyDeathsResp6MoFlu)
val deathBase: Seq[Int] = (1 to howManyDays).map( d => usa2017DailyDeaths)
val cancerBase: Seq[Int] = (1 to howManyDays).map( d => usa2017CancerDaily)
val heartBase: Seq[Int] = (1 to howManyDays).map( d => usa2017HeartDaily)
val accidentBase: Seq[Int] = (1 to howManyDays).map( d => usa2017AccidentDaily)
val suicideBase: Seq[Int] = (1 to howManyDays).map( d => usa2017SuicideDaily)



val trace0 = Scatter(
  (1 to howManyDays),
  deathBase,
  name = "2017 Daily Deaths, All Causes"
)

val trace01 = Scatter(
  (1 to howManyDays),
  cancerBase,
  name = "2017 Daily Deaths, Cancer"
)

val trace02 = Scatter(
  (1 to howManyDays),
  heartBase,
  name = "2017 Daily Deaths, Heart Disease"
)

val trace03 = Scatter(
  (1 to howManyDays),
  accidentBase,
  name = "2017 Daily Deaths, Accidents"
)

val trace04 = Scatter(
  (1 to howManyDays),
  suicideBase,
  name = "2017 Daily Deaths, Suidice"
)

val trace1 = Scatter(
  (1 to howManyDays),
  ipnBase,
  name = "2017 Daily Repiratory-related Deaths"
)

// usa2017DailyDeathsResp6MoFlu

val trace2 = Scatter(
  (1 to howManyDays),
  ipnBase6Mo,
  name = "2017 Daily Repiratory-related Deaths: 6-month Flu Season"

)

val trace3 = Scatter(
  (1 to howManyDays),
  deaths,
  name = "2020 Covid Daily Deaths",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(204, 204, 204, 0.95),
    line = Line(
      color = Color.RGBA(217, 217, 217, 1.0),
      width = 1.0
    ),
    symbol = Symbol.Circle(),
    size = 3
  )
)

val data = Seq(trace0, trace01, trace02, trace03, trace04, trace1, trace2, trace3)

val layout = Layout("Daily USA COVID Deaths vs. Other Causes")

plot(data, layout)

deaths: Seq[Int] = Vector(
  0,
...
howManyDays: Int = 77
ipnBase: Seq[Int] = Vector(
  591,
...
ipnBase6Mo: Seq[Int] = Vector(
  743,
...
deathBase: Seq[Int] = Vector(
  7708,
...
cancerBase: Seq[Int] = Vector(
  1641,
...
heartBase: Seq[Int] = Vector(
  1773,
...
accidentBase: Seq[Int] = Vector(
  465,
...
suicideBase: Seq[Int] = Vector(
  129,
...
trace0: Scatter = Scatter(
  Some(
...
trace01: Scatter = Scatter(
  Some(
...
trace02: Scatter = Scatter(
  Some(
...
trace03: Scatter = Scatter(
  Some(
...
trace04: Scatter = Scatter(
  Some(
...
trace1: Scatter = Scatter(
  Some(
...
trace2: Scatter = Scatter(
  Some(
...
trace3: Scatter = Scatter(
  Some(
...
data: Seq[Scatter] = List(
  Scatter(
...
layout: Layout = Layout(
  Some("Daily USA COVID Deaths vs. Other Causes"),
...
res15_19: String = "plot-711bad67-1c6d-42e8-907e-879c496786fb"

## South Carolina

Source for USA Data: [“Deaths and Mortality”, CDC](https://www.cdc.gov/nchs/fastats/deaths.htm).

Source for 2018 South Carolina Data: [“Stats of the State of South Carolina”, CDC](https://www.cdc.gov/nchs/pressroom/states/southcarolina/southcarolina.htm) and [SC DHEC](https://www.scdhec.gov/vital-records/parentage/sc-vital-records-data-and-statistics)

In [17]:
// Total Deaths
val sc2018TotalDeaths: Int = 50633
val sc2018DailyDeaths: Int = sc2018TotalDeaths / 365

// Chronic Lower Respiratory Disease
val sc2017TotalDeathsCLRD: Int = 2990
val sc2017DailyDeathsCLRD: Int = 2990 / 365

// Influenza and Pneumonia
val sc2017TotalDeathsIPn: Int = 882
val sc2017DailyDeathsIPn: Int = 882 / 365

// All Respiratory-Related Deaths
val sc2017DailyDeathsAllResp = sc2017DailyDeathsCLRD + sc2017DailyDeathsIPn

// Influenza and Pneumonia: 6 Month Flu Season
val sc2017DailyDeathsAllResp6Mo: Int = {
    sc2017DailyDeathsCLRD + (sc2017TotalDeathsIPn / 182 )
}

sc2018TotalDeaths: Int = 50633
sc2018DailyDeaths: Int = 138
sc2017TotalDeathsCLRD: Int = 2990
sc2017DailyDeathsCLRD: Int = 8
sc2017TotalDeathsIPn: Int = 882
sc2017DailyDeathsIPn: Int = 2
sc2017DailyDeathsAllResp: Int = 10
sc2017DailyDeathsAllResp6Mo: Int = 12

In [18]:
val deaths: Seq[Int] = totalNewDaily(state = Some("South Carolina")).map(_.newDeaths).toSeq

val howManyDays: Int = deaths.size
val ipnBase: Seq[Int] = (1 to howManyDays).map( d => sc2017DailyDeathsAllResp)
val ipnBase6Mo: Seq[Int] = (1 to howManyDays).map( d => sc2017DailyDeathsAllResp6Mo)
val deathBase: Seq[Int] = (1 to howManyDays).map( d => sc2018DailyDeaths)



val trace0 = Scatter(
  (1 to howManyDays),
  deathBase,
  name = "2018 Daily Deaths, All Causes"
)



val trace1 = Scatter(
  (1 to howManyDays),
  ipnBase,
  name = "2018 Daily Repiratory-related Deaths"
)

// usa2017DailyDeathsResp6MoFlu

val trace2 = Scatter(
  (1 to howManyDays),
  ipnBase6Mo,
  name = "2018 Daily Repiratory-related Deaths: 6-month Flu Season"

)

val trace3 = Scatter(
  (1 to howManyDays),
  deaths,
  name = "Covid Daily Deaths",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(204, 204, 204, 0.95),
    line = Line(
      color = Color.RGBA(217, 217, 217, 1.0),
      width = 1.0
    ),
    symbol = Symbol.Circle(),
    size = 3
  )
)

val data = Seq(trace0, trace1, trace2, trace3)

val layout = Layout("South Carolina: Daily COVID Deaths vs. Normal All-Respiratory-Related Deaths")

plot(data, layout)

deaths: Seq[Int] = Vector(
  0,
...
howManyDays: Int = 32
ipnBase: Seq[Int] = Vector(
  10,
...
ipnBase6Mo: Seq[Int] = Vector(
  12,
...
deathBase: Seq[Int] = Vector(
  138,
...
trace0: Scatter = Scatter(
  Some(
...
trace1: Scatter = Scatter(
  Some(
...
trace2: Scatter = Scatter(
  Some(
...
trace3: Scatter = Scatter(
  Some(
...
data: Seq[Scatter] = List(
  Scatter(
...
layout: Layout = Layout(
  Some(
...
res17_11: String = "plot-adc80f4a-708d-4681-98bf-c5cdede0e8ca"

## USA Case Fatality Rate

A running total of the *total deaths* divided by the *total official cases*.

In [26]:


val groupedDays: Vector[(String, Vector[StateDatum])] = rt.days.groupBy(_.date).toVector.sortBy(_._1)

case class DailyAggregate( cases: Int, deaths: Int)

val pilingUp: Vector[DailyAggregate] = groupedDays.map( d => {
    val dayString: String = d._1
    val dataVec: Vector[StateDatum] = d._2
    val sumCases: Int = dataVec.map(_.cases).sum
    val sumDeaths: Int = dataVec.map(_.deaths).sum
    DailyAggregate(sumCases, sumDeaths)
})

val cfr: Vector[Double] = pilingUp.map(d => {
    d.deaths.toDouble / d.cases.toDouble
})

val cfrSeq: Seq[Double] = cfr.toSeq

val howManyDays: Int = cfrSeq.size


val trace0 = Scatter(
  (1 to howManyDays),
  cfrSeq,
  name = "USA Case Fatality Rate",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(255, 0, 0, 1.0),
    line = Line(
      color = Color.RGBA(255, 0, 0, 1.0),
      width = 0.5
    ),
    symbol = Symbol.Circle(),
    size = 3
  )
)

val data = Seq(trace0)

val layout = Layout("USA Case Fatality Rate")

plot(data, layout)




groupedDays: Vector[(String, Vector[StateDatum])] = Vector(
  ("2020-01-21", Vector(StateDatum("2020-01-21", "Washington", 1, 0))),
...
defined class DailyAggregate
pilingUp: Vector[DailyAggregate] = Vector(
  DailyAggregate(1, 0),
...
cfr: Vector[Double] = Vector(
  0.0,
...
cfrSeq: Seq[Double] = Vector(
  0.0,
...
howManyDays: Int = 77
trace0: Scatter = Scatter(
  Some(
...
data: Seq[Scatter] = List(
  Scatter(
...
layout: Layout = Layout(
  Some("USA Case Fatality Rate"),
...
res25_9: String = "plot-dc137d0c-17a5-4000-9bc0-cfbcf853cbea"

## USA Cumulative Numbers

In [28]:
val totalCasesSeq: Seq[Int] = pilingUp.map(_.cases).toSeq

val totalDeathsSeq: Seq[Int] = pilingUp.map(_.deaths).toSeq


val trace0 = Scatter(
  (1 to howManyDays),
  totalCasesSeq,
  name = "Total Cases in USA",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(0, 0, 255, 1.0),
    line = Line(
      color = Color.RGBA(0, 0, 255, 1.0),
      width = 0.5
    ),
    symbol = Symbol.Circle(),
    size = 3
  )
)

val trace1 = Scatter(
  (1 to howManyDays),
  totalDeathsSeq,
  name = "Total Deaths in USA",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(255, 0, 0, 1.0),
    line = Line(
      color = Color.RGBA(255, 0, 0, 1.0),
      width = 0.5
    ),
    symbol = Symbol.Circle(),
    size = 3
  )
)



val data = Seq(trace0, trace1)

val layout = Layout("USA Totals")

plot(data, layout)





totalCasesSeq: Seq[Int] = Vector(
  1,
...
totalDeathsSeq: Seq[Int] = Vector(
  0,
...
trace0: Scatter = Scatter(
  Some(
...
trace1: Scatter = Scatter(
  Some(
...
data: Seq[Scatter] = List(
  Scatter(
...
layout: Layout = Layout(
  Some("USA Totals"),
...
res27_6: String = "plot-4f7c2b0d-3b7a-441d-9bfe-6987d91f2f46"

## Georgia

Source for USA Data: [“Deaths and Mortality”, CDC](https://www.cdc.gov/nchs/fastats/deaths.htm).

Source for 2017 Georgia Data: [“Stats of the State of South Carolina”, CDC](hhttps://www.cdc.gov/nchs/pressroom/states/georgia/georgia.htm), and [OASIS, Georgia Department of Public Health](https://oasis.state.ga.us/oasis/webquery/qryMortality.aspx).

In [29]:
// Total Deaths
val ga2018TotalDeaths: Int = 83122
val ga2018DailyDeaths: Int = ga2018TotalDeaths / 365



// All Respiratory-Related Deaths
val ga2017DailyDeathsAllResp = 8526


ga2018TotalDeaths: Int = 83122
ga2018DailyDeaths: Int = 227
ga2017DailyDeathsAllResp: Int = 8526

In [31]:
val gaDeaths: Seq[Int] = totalNewDaily(state = Some("Georgia")).map(_.newDeaths).toSeq

val howManyDays: Int = deaths.size
val gaAllDeathsBase: Seq[Int] = (1 to howManyDays).map( d => sc2017DailyDeathsAllResp)
val gaRespDeaths: Seq[Int] = (1 to howManyDays).map( d => sc2018DailyDeaths)



val trace0 = Scatter(
  (1 to howManyDays),
  gaAllDeathsBase,
  name = "2017 Daily Deaths, All Causes"
)



val trace1 = Scatter(
  (1 to howManyDays),
  gaRespDeaths,
  name = "2017 Daily Deaths, All Respiratory Disease"
)



val trace3 = Scatter(
  (1 to howManyDays),
  gaDeaths,
  name = "Covid Daily Deaths",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(204, 204, 204, 0.95),
    line = Line(
      color = Color.RGBA(217, 217, 217, 1.0),
      width = 1.0
    ),
    symbol = Symbol.Circle(),
    size = 3
  )
)

val data = Seq(trace0, trace1, trace2, trace3)

val layout = Layout("Georgia: Daily COVID Deaths vs. Normal All-Respiratory-Related Deaths")

plot(data, layout)

gaDeaths: Seq[Int] = Vector(
  0,
...
howManyDays: Int = 32
gaAllDeathsBase: Seq[Int] = Vector(
  10,
...
gaRespDeaths: Seq[Int] = Vector(
  138,
...
trace0: Scatter = Scatter(
  Some(
...
trace1: Scatter = Scatter(
  Some(
...
trace3: Scatter = Scatter(
  Some(
...
data: Seq[Scatter] = List(
  Scatter(
...
layout: Layout = Layout(
  Some("Georgia: Daily COVID Deaths vs. Normal All-Respiratory-Related Deaths"),
...
res30_9: String = "plot-d43cfe71-9d95-44ae-85d4-0cf62bb93164"